In [5]:
import os
os.chdir('readydemo')

# ML Ops with Azure ML and Azure DevOps

1. Create a training pipeline with Azure ML. 
2. Publish this pipeline so it can be used to control and automate the training process - including retraining later on.
3. Use Azure DevOps to automate the release of your model once it is ready for E2E deployment.

Problems to solve (DS):
1. tracking my work
2. iterating quickly as I experiment (and collaborate across my team)
3. comparing and evaluating via leaderboards

Problems to solve (DevOps):
1. model reproducibility
2. model valdidatio
3. model versioning
4. model deployment



In [3]:
!pip install gitpython

    100% |████████████████████████████████| 450kB 28.0MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 31.6MB/s ta 0:00:01


In [3]:
from azureml.core import Workspace, Datastore
from azureml.core.compute import AmlCompute, DataFactoryCompute
from azureml.core.runconfig import CondaDependencies, RunConfiguration
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.core.graph import PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.steps import DataTransferStep

import git

In [6]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /data/home/demo/notebooks/readydemo/config.json
contosomanufacturing
scottgu-all-hands
eastus2
2a779d6f-0806-4359-a6e8-f1fd57bb5dd7


# Declare resources you want to use:
- Computes
- Datastores (and data sets)
- Configuration for training (Container Images / Conda Dependencies you want to use)

In [12]:
aml_compute_target = "cpu"
data_factory_name = "adf"
default_dataset = "soda_cans_training_data"
project_folder = "./mobilenetscripts"

ds = ws.get_default_datastore()
source_ds = Datastore.get(ws, 'amlvdaik14969151586')

# Declare packages dependencies required in the pipeline (these can also be expressed as a YML file)
cd = CondaDependencies.create(pip_packages=["azureml-defaults", 'tensorflow==1.8.0'])
amlcompute_run_config = RunConfiguration(conda_dependencies=cd)

# Define our computes
data_factory_compute = DataFactoryCompute(ws, data_factory_name)
aml_compute = AmlCompute(ws, aml_compute_target)

# Define datasets you want to use

In [13]:

# We explicitly declare the data we're using in this training pipeline
source_images = DataReference(datastore=source_ds,
                              data_reference_name="original_images",
                              path_on_datastore=default_dataset)
dest_images = DataReference(datastore=ds,
                            data_reference_name="transferred_images",
                            path_on_datastore='training_images')
mlops = DataReference(datastore=ds,
                      data_reference_name="mlops_connector",
                      path_on_datastore='mlops')
training_dataset = DataPath(datastore=source_ds, path_on_datastore=default_dataset)

# Define pipeline parameters

In [14]:
# Parameters make it easy for us to re-run this training pipeline, including for retraining.
model_variant = PipelineParameter(name="model_variant", default_value='sodacans')
training_dataset_param = (PipelineParameter(name="training_dataset",
                                            default_value=training_dataset),
                          DataPathComputeBinding())


# We pass the trained model from the transfer learning step to the model registration step
model = PipelineData(name="model", datastore=ds, output_path_on_compute="model")
model_id = PipelineData(name="modelId", datastore=ds)


# Define pipeline steps

Pipeline steps are defined for:
1. transferring and copying data
2. training model (via transfer learning)
3. Evaluating and registering the model to kick off the CI/CD process.

In [ ]:
# Copying data into a datastore we manage ensures we can reproduce the model later on.
datatransfer = DataTransferStep(
    name="Copy training data for improved performance and model reproducibility",
    source_data_reference=source_images,
    destination_data_reference=dest_images,
    compute_target=data_factory_compute)


# You'll note this is similar to the code from the notebook.
# We've done some cleanup to reflect the proper parameterization of the steps.



train = PythonScriptStep(name="Train new model via transfer learning",
                         script_name="train.py",
                         compute_target=aml_compute,
                         runconfig=amlcompute_run_config,
                         inputs=[training_dataset_param, dest_images],
                         outputs=[model],
                         source_directory=project_folder,
                         arguments=['--image_dir', training_dataset_param,
                                    '--architecture', 'mobilenet_1.0_224',
                                    '--output_dir', model,
                                    '--output_graph', 'retrained_graph.pb',
                                    '--output_labels', 'output_labels.txt',
                                    '--model_file_name', 'imagenet_2_frozen.pb'
                                   ])



register = PythonScriptStep(name="Register model for deployment",
                            script_name="register.py",
                            compute_target=aml_compute,
                            inputs=[model, mlops],
                            arguments=['--dataset_name', model_variant,
                                       '--model_assets_path', model
                                      ],
                            outputs=[model_id],
                            source_directory=project_folder)



# Create Pipeline

In [16]:
steps = [datatransfer, train, register]
pipeline = Pipeline(workspace=ws, steps=steps)
pipeline.validate()

Step Train new model via transfer learning is ready to be created [9275a21e]
Step Register model for deployment is ready to be created [146610f4]
Data reference amlvdaik14969151586_e538dadb_f05f16b4 is ready to be created [7104c593], (Consumers of this data will generate new runs.)


[]

# Version your Code via Git Integration

Git integration is one of our key upcoming mlops investment areas.

We are planning to support first-class tracking of code (for pipeline scripts, pipeline management, inference code management).

Tagging our training pipeline with repo/commit/branch information helps us flesh out the E2E audit trail for code (we can easily diff scripts between pipelines).

In [ ]:
repo = git.Repo(search_parent_directories=True)
tags = {
    'git.repo': repo.remotes.origin.url,
    'git.commit': repo.head.object.hexsha,
    'git.branch': repo.active_branch.name
}

# Publish Training Pipeline

In [ ]:

mlpipeline = pipeline.publish(name="Ready ML Training Pipeline",
                              description="Retrain a mobilenet.imagenet model.")

print("Pipeline Published ID:"+mlpipeline.id)

# Submit a Training Job to Classify Soda Cans

In [ ]:
mlpipeline.submit(ws, "sodacanclassifier",
                  pipeline_parameters={"training_dataset":DataPath(datastore=source_ds,
                                                                   path_on_datastore="soda_cans_training_data"),
                                       "model_variant":"sodacans"}).set_tags(tags)

Your training pipeline experiment has been submitted successfully.

[Click here to view your training experiment](https://ms.portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourceGroups/DevOps_AzureML_Demo/providers/Microsoft.MachineLearningServices/workspaces/AzureML_Demo_ws/overview)

# View CI/CD Pipeline for Your Model

Azure DevOps, along with the Azure ML CLI, is used to automate the E2E release of your model.

You can view the release of your model [here](https://dev.azure.com/aidemos/DevOpsAIDemo/_releaseProgress?_a=release-pipeline-progress&releaseId=22)